In [1]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from ...server.automl.neural_network.neural_network import NeuralNetwork
from ...server.automl.neural_network.fc_layer import FCLayer
from ...server.automl.neural_network.activation_layer import ActivationLayer
from ...server.automl.neural_network.activation_functions import tanh, tanh_derivative, relu, relu_derivative
from ...server.automl.neural_network.loss_functions import mse, mse_derivative
from ray import tune
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.suggest.bayesopt import BayesOptSearch
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.neural_network import MLPRegressor

In [2]:
#Lodaing data
X, Y = load_diabetes(return_X_y=True)
print(X.shape, Y.shape)
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.2)
#Normalizing data
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(442, 10) (442,)
(353, 10) (89, 10) (353,) (89,)


In [ ]:
network = NeuralNetwork()
network.add(FCLayer(X_train.shape[1], 5))
network.add(ActivationLayer(tanh, tanh_derivative))
network.add(FCLayer(5, 2))
network.add(ActivationLayer(tanh, tanh_derivative))
network.add(FCLayer(2, 1))
network.use(mse, mse_derivative)
network.fit(X_train, y_train, epochs=2000, learning_rate=0.1)
y_pred = np.array(network.predict(X_test))
print(y_test, y_test.shape)
print(y_pred, y_pred.shape)
print(mean_squared_error(y_test, y_pred))

In [ ]:
clf = MLPRegressor(solver='lbfgs', alpha=0.1,
                                            hidden_layer_sizes=(5,2),
                                            max_iter=2000,activation='relu')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(mean_squared_error(y_test, y_pred))

In [3]:
#Function For BayesOpt
def my_func(config, reporter):  # add the reporter parameter
  # report the configuration
  network = NeuralNetwork()
  network.add(FCLayer(X_train.shape[1], 5))
  network.add(ActivationLayer(tanh, tanh_derivative))
  network.add(FCLayer(5, 2))
  network.add(ActivationLayer(tanh, tanh_derivative))
  network.add(FCLayer(2, 1))
  network.use(mse, mse_derivative)
  network.fit(X_train, y_train, epochs=1000, learning_rate=config["alpha"])
  y_pred = np.array(network.predict(X_test))
  print(y_pred)
  reporter(config, mean_loss = mean_squared_error(y_test, y_pred))

In [4]:
#Bayesian Optimization
algo = BayesOptSearch(utility_kwargs={"kind": "ucb", "kappa": 2.5, "xi": 0.0})
algo = ConcurrencyLimiter(algo, max_concurrent=4)
scheduler = AsyncHyperBandScheduler()
analysis = tune.run(
        my_func,
        name="my_exp",
        metric="mean_loss",
        stop={"mean_loss": 3000},
        mode="min",
        search_alg=algo,
        scheduler=scheduler,
        num_samples=10,
        config={
            "steps": 100,
            "input_dim": X_train.shape[1],
            "hidden_dim": 5,
            
            "output_dim": 1,
            "alpha": tune.uniform(0.0001, 10),
        },
    )

print("Best hyperparameters found were: ", analysis.best_config)

c:\Users\Public\anaconda3\lib\site-packages\ray\_private\services.py:238: UserWarning: Not all Ray Dashboard dependencies were found. To use the dashboard please install Ray using `pip install ray[default]`. To disable this message, set RAY_DISABLE_IMPORT_WARNING env var to '1'.
  warnings.warn(warning_message)
2022-05-10 05:32:30,525	WARNING bayesopt.py:397 -- BayesOpt does not support specific sampling methods. The Uniform sampler will be dropped.


Trial name,status,loc,alpha
my_func_d27c419d,PENDING,,3.74546


2022-05-10 05:32:30,768	ERROR syncer.py:72 -- Log sync requires rsync to be installed.
(pid=16564) c:\Users\Mohamed Ghallab\Desktop\GP\AutoML\notebooks\neural_network_generation\loss_functions.py:6: RuntimeWarning: overflow encountered in power
(pid=16564)   return np.mean(np.power(y_true-y_pred, 2));
(pid=16564) c:\Users\Mohamed Ghallab\Desktop\GP\AutoML\notebooks\neural_network_generation\activation_layer.py:21: RuntimeWarning: invalid value encountered in multiply
(pid=16564)   return self.activation_function_derivative(self.input) * output_error
(pid=18516) c:\Users\Mohamed Ghallab\Desktop\GP\AutoML\notebooks\neural_network_generation\activation_layer.py:21: RuntimeWarning: invalid value encountered in multiply
(pid=18516)   return self.activation_function_derivative(self.input) * output_error
(pid=15460) c:\Users\Mohamed Ghallab\Desktop\GP\AutoML\notebooks\neural_network_generation\loss_functions.py:6: RuntimeWarning: overflow encountered in power
(pid=15460)   return np.mean(np.p

Result for my_func_d27c419d:
  {}
  


(pid=16564) 2022-05-10 05:33:39,942	ERROR function_runner.py:266 -- Runner Thread raised error.
(pid=16564) Traceback (most recent call last):
(pid=16564)   File "c:\Users\Public\anaconda3\lib\site-packages\ray\tune\function_runner.py", line 260, in run
(pid=16564)     self._entrypoint()
(pid=16564)   File "c:\Users\Public\anaconda3\lib\site-packages\ray\tune\function_runner.py", line 328, in entrypoint
(pid=16564)     return self._trainable_func(self.config, self._status_reporter,
(pid=16564)   File "c:\Users\Public\anaconda3\lib\site-packages\ray\tune\function_runner.py", line 594, in _trainable_func
(pid=16564)     output = fn()
(pid=16564)   File "<ipython-input-3-d116da951592>", line 14, in my_func
(pid=16564)   File "c:\Users\Public\anaconda3\lib\site-packages\sklearn\utils\validation.py", line 63, in inner_f
(pid=16564)     return f(*args, **kwargs)
(pid=16564)   File "c:\Users\Public\anaconda3\lib\site-packages\sklearn\metrics\_regression.py", line 335, in mean_squared_error
(p

...

(pid=15460) 2022-05-10 05:33:41,095	ERROR function_runner.py:266 -- Runner Thread raised error.
(pid=15460) Traceback (most recent call last):
(pid=15460)   File "c:\Users\Public\anaconda3\lib\site-packages\ray\tune\function_runner.py", line 260, in run
(pid=15460)     self._entrypoint()
(pid=15460)   File "c:\Users\Public\anaconda3\lib\site-packages\ray\tune\function_runner.py", line 328, in entrypoint
(pid=15460)     return self._trainable_func(self.config, self._status_reporter,
(pid=15460)   File "c:\Users\Public\anaconda3\lib\site-packages\ray\tune\function_runner.py", line 594, in _trainable_func
(pid=15460)     output = fn()
(pid=15460)   File "<ipython-input-3-d116da951592>", line 14, in my_func
(pid=15460)   File "c:\Users\Public\anaconda3\lib\site-packages\sklearn\utils\validation.py", line 63, in inner_f
(pid=15460)     return f(*args, **kwargs)
(pid=15460)   File "c:\Users\Public\anaconda3\lib\site-packages\sklearn\metrics\_regression.py", line 335, in mean_squared_error
(p

(pid=15460) [[nan]
(pid=15460)  [nan]
(pid=15460)  [nan]
(pid=15460)  [nan]
(pid=15460)  [nan]
(pid=15460)  [nan]
(pid=15460)  [nan]
(pid=15460)  [nan]
(pid=15460)  [nan]
(pid=15460)  [nan]
(pid=15460)  [nan]
(pid=15460)  [nan]
(pid=15460)  [nan]
(pid=15460)  [nan]
(pid=15460)  [nan]
(pid=15460)  [nan]
(pid=15460)  [nan]
(pid=15460)  [nan]
(pid=15460)  [nan]
(pid=15460)  [nan]
(pid=15460)  [nan]
(pid=15460)  [nan]
(pid=15460)  [nan]
(pid=15460)  [nan]
(pid=15460)  [nan]
(pid=15460)  [nan]
(pid=15460)  [nan]
(pid=15460)  [nan]
(pid=15460)  [nan]
(pid=15460)  [nan]
(pid=15460)  [nan]
(pid=15460)  [nan]
(pid=15460)  [nan]
(pid=15460)  [nan]
(pid=15460)  [nan]
(pid=15460)  [nan]
(pid=15460)  [nan]
(pid=15460)  [nan]
(pid=15460)  [nan]
(pid=15460)  [nan]
(pid=15460)  [nan]
(pid=15460)  [nan]
(pid=15460)  [nan]
(pid=15460)  [nan]
(pid=15460)  [nan]
(pid=15460)  [nan]
(pid=15460)  [nan]
(pid=15460)  [nan]
(pid=15460)  [nan]
(pid=15460)  [nan]
(pid=15460)  [nan]
(pid=15460)  [nan]
(pid=15460) 

2022-05-10 05:33:41,349	ERROR trial_runner.py:773 -- Trial my_func_d2bc7340: Error processing event.
Traceback (most recent call last):
  File "c:\Users\Public\anaconda3\lib\site-packages\ray\tune\trial_runner.py", line 739, in _process_trial
    results = self.trial_executor.fetch_result(trial)
  File "c:\Users\Public\anaconda3\lib\site-packages\ray\tune\ray_trial_executor.py", line 746, in fetch_result
    result = ray.get(trial_future[0], timeout=DEFAULT_GET_TIMEOUT)
  File "c:\Users\Public\anaconda3\lib\site-packages\ray\_private\client_mode_hook.py", line 82, in wrapper
    return func(*args, **kwargs)
  File "c:\Users\Public\anaconda3\lib\site-packages\ray\worker.py", line 1621, in get
    raise value.as_instanceof_cause()
ray.exceptions.RayTaskError(TuneError): ray::ImplicitFunc.train_buffered() (pid=2688, ip=192.168.1.105, repr=<types.ImplicitFunc object at 0x0000014BD3FC21C0>)
  File "python\ray\_raylet.pyx", line 536, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx

Result for my_func_d2bc7340:
  {}
  
(pid=2688) [[nan]
(pid=2688)  [nan]
(pid=2688)  [nan]
(pid=2688)  [nan]
(pid=2688)  [nan]
(pid=2688)  [nan]
(pid=2688)  [nan]
(pid=2688)  [nan]
(pid=2688)  [nan]
(pid=2688)  [nan]
(pid=2688)  [nan]
(pid=2688)  [nan]
(pid=2688)  [nan]
(pid=2688)  [nan]
(pid=2688)  [nan]
(pid=2688)  [nan]
(pid=2688)  [nan]
(pid=2688)  [nan]
(pid=2688)  [nan]
(pid=2688)  [nan]
(pid=2688)  [nan]
(pid=2688)  [nan]
(pid=2688)  [nan]
(pid=2688)  [nan]
(pid=2688)  [nan]
(pid=2688)  [nan]
(pid=2688)  [nan]
(pid=2688)  [nan]
(pid=2688)  [nan]
(pid=2688)  [nan]
(pid=2688)  [nan]
(pid=2688)  [nan]
(pid=2688)  [nan]
(pid=2688)  [nan]
(pid=2688)  [nan]
(pid=2688)  [nan]
(pid=2688)  [nan]
(pid=2688)  [nan]
(pid=2688)  [nan]
(pid=2688)  [nan]
(pid=2688)  [nan]
(pid=2688)  [nan]
(pid=2688)  [nan]
(pid=2688)  [nan]
(pid=2688)  [nan]
(pid=2688)  [nan]
(pid=2688)  [nan]
(pid=2688)  [nan]
(pid=2688)  [nan]
(pid=2688)  [nan]
(pid=2688)  [nan]
(pid=2688)  [nan]
(pid=2688)  [nan]
(pid=2688

(pid=19800) c:\Users\Mohamed Ghallab\Desktop\GP\AutoML\notebooks\neural_network_generation\loss_functions.py:6: RuntimeWarning: overflow encountered in power
(pid=19800)   return np.mean(np.power(y_true-y_pred, 2));
(pid=19800) c:\Users\Mohamed Ghallab\Desktop\GP\AutoML\notebooks\neural_network_generation\activation_layer.py:21: RuntimeWarning: invalid value encountered in multiply
(pid=19800)   return self.activation_function_derivative(self.input) * output_error
(pid=21516) c:\Users\Mohamed Ghallab\Desktop\GP\AutoML\notebooks\neural_network_generation\loss_functions.py:6: RuntimeWarning: overflow encountered in power
(pid=21516)   return np.mean(np.power(y_true-y_pred, 2));
(pid=21516) c:\Users\Mohamed Ghallab\Desktop\GP\AutoML\notebooks\neural_network_generation\activation_layer.py:21: RuntimeWarning: invalid value encountered in multiply
(pid=21516)   return self.activation_function_derivative(self.input) * output_error
(pid=23780) c:\Users\Mohamed Ghallab\Desktop\GP\AutoML\noteboo